In [1]:
from tkinter import N
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup
import pandas as pd
import csv
import requests
import time
from datetime import datetime
import logging
from timeit import default_timer as timer  
import pandas as pd
import wikipedia
import numpy as np
import pycountry


In [2]:
habitats_list = ['forest', 'forests', 'woods', 'woods', 'woodland', 'woodlands', 'bog', 'bogs', 'fen', 'fens', 'marsh', 'marshes', 'swamp', 'swamps', 'wetland', 'grasslands', 'prairie', 'scrubs','shrubs', 'backyards']
def get_conservation_status(df1):
    for i in range(0,len(list(df1.index))):
        if list(df1.index)[i] == 'Conservation status':
            return (list(df1.index)[i+1])
    if 'Conservation status' not in list(df1.index):
        return np.NaN
def get_kingdom(df1):
    if 'Kingdom:' in list(df1.index):
        return (df1.loc['Kingdom:'][0])
    else:
        return np.NaN
def get_phylum(df1):
    if 'Phylum:' in list(df1.index):
        return (df1.loc['Phylum:'][0])
    else:
        return np.NaN
def get_class(df1):
    if 'Class:' in list(df1.index):
        return (df1.loc['Class:'][0])
    else:
        return np.NaN
def get_order(df1):
    if 'Order:' in list(df1.index):
        return (df1.loc['Order:'][0])
    else:
        return np.NaN
def get_family(df1):
    if 'Family:' in list(df1.index):
        return (df1.loc['Family:'][0])
    else:
        return np.NaN
def get_genus(df1):
    if 'Genus:' in list(df1.index):
        return (df1.loc['Genus:'][0])
    else:
        return np.NaN
def get_species(df1):
    if 'Species:' in list(df1.index):
        return (df1.loc['Species:'][0])
    else:
        return np.NaN
def get_binomial_name(df1):
    for i in range(0,len(list(df1.index))):
        if list(df1.index)[i] == 'Binomial name':
            return(list(df1.index)[i+1])
    if 'Binomial name' not in list(df1.index):
        return np.NaN
def get_synonyms(df1):
    for i in range(0,len(list(df1.index))):
        if list(df1.index)[i] == 'Synonyms':
            return(list(df1.index)[i+1])
    if 'Synonyms' not in list(df1.index):
        return np.NaN
def get_summary(name):
    return(wikipedia.summary(name, sentences=2))
def get_categories(name):
    categories = wikipedia.page(name)
    return(categories.categories[0:3])
def get_references(name):
    refs = wikipedia.page(name)
    return(refs.references[0:3])
def get_url(name):
    url = wikipedia.page(name)
    return(url.url)
def get_image(name):
    image = wikipedia.page(name)
    return(image.images[0])
def findCountry(data):
    countries = sorted([country.name for country in pycountry.countries] , key=lambda x: -len(x))
    for country in countries:
        if country.lower() in data.lower():
            return country
    return None
def findHabitat(data):
    for habitat in habitats_list:
        if habitat in data.lower():
            return habitat
    return None
def extract_data(infobox):
    df = pd.DataFrame(infobox)
    df = df.dropna()
    row = [list(df.columns)[0]]
    
    df = df.set_index(list(df.columns)[0])
    
    row.append(get_conservation_status(df))
    row.append(get_kingdom(df))
    row.append(get_phylum(df))
    row.append(get_class(df))
    row.append(get_order(df))
    row.append(get_family(df))
    row.append(get_genus(df))
    row.append(get_species(df))
    row.append(get_binomial_name(df))
    row.append(get_synonyms(df))
    row.append(get_summary(row[0]))
    row.append(get_categories(row[0]))
    row.append(get_references(row[0]))
    row.append(get_url(row[0]))
    row.append(get_image(row[0]))
    
    birds_length = len(birds)
    birds.loc[birds_length] = row

In [3]:
startTime = datetime.now()
driver = webdriver.Chrome('./chromedriver.exe')

C:\Users\LEGION\AppData\Local\Temp\ipykernel_24316\2716060403.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')


In [4]:
types_of_birds_df = pd.read_csv('assets/wiki_list_to.csv')
list_of_bird_names = [ types_of_birds_df.loc[ _ ,'Common Name'] for _ in range(1,types_of_birds_df.shape[0]) ]
# f = 1
# t = len(list_of_bird_names)
# # f = 235
# # t = 237
# list_of_bird_names = list_of_bird_names[f:t]

In [5]:
# bird = 'Arabian scops owl'
# driver = webdriver.Chrome('./chromedriver.exe')
def get_page_of_bird(bird_name,driver):
    wiki = 'https://wikipedia.org/'
    driver.get(wiki)
    input_wiki = WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[name='search']")))
    input_wiki.send_keys(bird_name)
    input_wiki.send_keys(Keys.ENTER)
    bird_url = driver.current_url
    l = bird_url.lower()
    if bird_url != wiki and 'search' not in l :
        return bird_url
    return None
# get_page_of_bird('ashjvcasjh ajhdfbasj',driver)


In [6]:
birds = pd.DataFrame(columns=['Name','Conservation status','Kingdom','Phylum','Class','Order','Family', 'Genus', 'Species', 'Binomial Name', 'Synonyms', 'Summary', 'Categories', 'References', 'Wiki URL', 'Image'])

In [7]:
pages = []
for i in list_of_bird_names:
    link = get_page_of_bird(i,driver)
    if link != None:
        pages.append(link)

len(pages)

In [ ]:
driver.close()
pages_df = pd.DataFrame(pages)
pages_df.to_pickle('pages.pkl')

In [ ]:
failed = []
for i in pages:
    infoboxes = pd.read_html(i, attrs={"class":"infobox biota"})
    if len(infoboxes)==1:
        try:
            extract_data(infoboxes[0])
        except Exception:
            failed.append(i)

birds.head(5)

,Name,Conservation status,Kingdom,Phylum,Class,Order,Family,Genus,Species,Binomial Name,Synonyms,Summary,Categories,References,Wiki URL,Image
0,Cinnamon weaver,Least Concern (IUCN 3.1)[1],Animalia,Chordata,Aves,Passeriformes,Ploceidae,Ploceus,P. badius,"Ploceus badius(Cassin, 1850)",NaN,The cinnamon weaver (Ploceus badius) is a spec...,"[All stub articles, Articles with 'species' mi...",[http://www.hbw.com/species/cinnamon-weaver-pl...,https://en.wikipedia.org/wiki/Cinnamon_weaver,https://upload.wikimedia.org/wikipedia/commons...


In [ ]:
# !pip install pycountry
birds['Countries'] = np.NaN
birds['Habitat'] = np.NaN

In [ ]:
for i in range(0, len(birds)):
    name = birds.Name[i]
    summary = wikipedia.summary(name)
    birds.Countries[i] = findCountry(summary)
    birds.Habitat[i] = findHabitat(summary)
birds

C:\Users\LEGION\AppData\Local\Temp\ipykernel_23528\1130164129.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  birds.Countries[i] = findCountry(summary)
C:\Users\LEGION\AppData\Local\Temp\ipykernel_23528\1130164129.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  birds.Habitat[i] = findHabitat(summary)


,Name,Conservation status,Kingdom,Phylum,Class,Order,Family,Genus,Species,Binomial Name,Synonyms,Summary,Categories,References,Wiki URL,Image,Countries,Habitat
0,Cinnamon weaver,Least Concern (IUCN 3.1)[1],Animalia,Chordata,Aves,Passeriformes,Ploceidae,Ploceus,P. badius,"Ploceus badius(Cassin, 1850)",NaN,The cinnamon weaver (Ploceus badius) is a spec...,"[All stub articles, Articles with 'species' mi...",[http://www.hbw.com/species/cinnamon-weaver-pl...,https://en.wikipedia.org/wiki/Cinnamon_weaver,https://upload.wikimedia.org/wikipedia/commons...,South Sudan,NaN


In [ ]:
ds = 'wiki_'+str(f)+'_'+str(t)
logging.basicConfig(filename="assets/rescrape_wiki/Log_"+ds+".log",format='%(asctime)s - %(levelname)s - %(message)s',filemode='a')
birds.to_pickle("assets/rescrape_wiki/Pick_"+ds+".pkl")
birds.to_csv("assets/rescrape_wiki/csv_"+ds+".csv")
birds.to_csv("assets/rescrape_wiki/failed_"+ds+".csv")
logging.info('\n\nTime taken : ',datetime.now() - startTime)
print('\n\nTime taken : ',datetime.now() - startTime) 



Time taken :  0:00:46.777429


In [ ]:
birds = birds.drop('Synonyms',axis = 1)
birds = birds.dropna(thresh=3, axis = 0)
birds.head(5)

,Name,Conservation status,Kingdom,Phylum,Class,Order,Family,Genus,Species,Binomial Name,Summary,Categories,References,Wiki URL,Image,Countries,Habitat
0,Cinnamon weaver,Least Concern (IUCN 3.1)[1],Animalia,Chordata,Aves,Passeriformes,Ploceidae,Ploceus,P. badius,"Ploceus badius(Cassin, 1850)",The cinnamon weaver (Ploceus badius) is a spec...,"[All stub articles, Articles with 'species' mi...",[http://www.hbw.com/species/cinnamon-weaver-pl...,https://en.wikipedia.org/wiki/Cinnamon_weaver,https://upload.wikimedia.org/wikipedia/commons...,South Sudan,NaN


In [ ]:
def clean_Name(df):
    sub_list = ["Conservation", "status", "Scientific", "classification"]
    for i in range(0, len(df)):
        for sub in sub_list:
            df.Name[i] = df.Name[i].replace(sub, ' ')
        df.Name[i] = df.Name[i].translate(str.maketrans('', '',string.punctuation))
        df.Name[i] = df.Name[i].strip()
        print(df.Name[i])
birds['Conservation status'] = birds['Conservation status'].astype(str)

In [ ]:
def clean_Conservation_Status(df):
    for i in range(0, len(df)): 
        df['Conservation status'][i] = df['Conservation status'][i].split('(',1)[0]
        print(df['Conservation status'][i])

In [ ]:
def clean_Categories_and_References(df):
    remove_list = '[]\"\''
    for i in range(0, len(df)):
        df.Categories[i] = df.Categories[i].translate(str.maketrans('', '', remove_list))
        df.References[i] = df.References[i].translate(str.maketrans('', '', remove_list))
        print(df.Categories[i])
        print(df.References[i])

In [ ]:
# clean_Categories_and_References(birds)
clean_Conservation_Status(birds)
# clean_Name(birds)

file_name = 'assets/rescrape_wiki/bird_wiki_dataset.xlsx'

Least Concern 


C:\Users\LEGION\AppData\Local\Temp\ipykernel_23528\255890669.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Conservation status'][i] = df['Conservation status'][i].split('(',1)[0]


In [ ]:
birds.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.
